# Finding stars behind SNRs

## Importing Files

In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import astropy
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii
from astropy.table import Table
from astropy.table import QTable
from astropy.io.votable import parse
from astropy.io import fits
import scipy as sp
import csv
%matplotlib inline

In [2]:
allStar_dir = '/uufs/chpc.utah.edu/common/home/uuastro/astro_data/zasowski/catalogs/allStar/allStar-r12-l33beta.fits'
stellar_data_dir = '/uufs/chpc.utah.edu/common/home/sdss11/dr16/apogee/spectro/aspcap/r12/l33/'
star_hdus = fits.open(allStar_dir)
print(star_hdus[1])
star_list = star_hdus[1].data
star_headers = star_hdus[1].header
star_hdus.close()
# star_headers

In [3]:
### Implementing quality cuts on allStar (early, so as to reduce runtime)
gd = np.logical_and.reduce([star_list['ASPCAPFLAG'] & 2**19 == 0,
                            star_list['ASPCAPFLAG'] & 2**20 == 0,
                            star_list['ASPCAPFLAG'] & 2**23 == 0,
                            star_list['STARFLAG'] & 2**9 ==0,
                            star_list['EXTRATARG'] == 0,
                            star_list['SNR'] > 60.,
                            np.array(star_list['LOGG']) > -0.75,
                            np.array(star_list['LOGG']) <= 3.5,
                            np.array(star_list['TEFF']) >= 3600.,
                            np.array(star_list['TEFF']) <= 4800.,
                            np.array(star_list['M_H']) > -5.0,
                            np.array(star_list['ALPHA_M']) > -5.0])

good_indices = [i for i, x in enumerate(gd) if x == True]

In [4]:
star_distances_dir = '/uufs/astro.utah.edu/common/uuastro/astrougrad/u1068803/SNR_Thesis/cbj_spectroPhotom_allStar_dr16beta_order.fits'
star_dis_hdus = fits.open(star_distances_dir)
print(star_dis_hdus[1])
star_dis_list = star_dis_hdus[1].data
star_dis_headers = star_dis_hdus[1].header
star_dis_hdus.close()
#star_dis_headers

In [5]:
SNR_catalog_dir = '/uufs/astro.utah.edu/common/uuastro/astrougrad/u1068803/SNR_Thesis/GreenSNR.vot'
SNR_images = '/uufs/astro.utah.edu/common/uuastro/astrougrad/u1068803/SNR_Thesis/Stars_in_SNRs/'
votable = parse(SNR_catalog_dir)
GreensCat = votable.get_first_table().to_table()

### Importing the SNR distance estimates I pulled from Green's website
green_dist = '/uufs/astro.utah.edu/common/uuastro/astrougrad/u1068803/SNR_Thesis/SNR_distances.csv'
SNR_dist = []
with open(green_dist) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[1] == '':
            SNR_dist.append(np.NaN)
        else:
            SNR_dist.append(float(row[1]))

### Going through Green's Cat. and creating images of stars within 2x radius.

In [17]:
### Defining arrays to store values in for each star,
### which will then be stored in a .fits file so the search only
### has to be done once.
GREEN_ID = []
REMNANT = []
TELESCOPE = []
FIELD = []
APOGEE_ID = []
RA = [] #* u.deg
DEC = [] #* u.deg
SNR = []
TEFF = []
LOGG = []
M_H = []
ALPHA_M = []
RV = []
ASPCAPFLAG = []
STARFLAG = []
EXTRATARG = []
ANG_SEP = [] # Angular separation between star and remnant center
PHYS_SEP = [] # Physical separation in plane of the remnant/sky
DIST = [] # Distance to the star

In [7]:
SNR_coord = SkyCoord(GreensCat['RAJ2000'], GreensCat['DEJ2000'], unit=(u.hourangle, u.degree))
star_coord = SkyCoord(star_list['ra'], star_list['dec'], unit=(u.degree, u.degree))
good_star_coord = SkyCoord(star_list['ra'][good_indices], star_list['dec'][good_indices], unit=(u.degree, u.degree))

In [18]:
plt.ioff()
for g in range(len(GreensCat)):
    if np.isnan(SNR_dist[g]) == False:
        dist_count = 0
        g_SNR = GreensCat[g]
        g_title = str(g) + '_' + g_SNR['SNR']
        g_rad = float(g_SNR['Dmaj']/60 * 0.5)     

        ### Finding stars within 2x the Green's radius
        g_internal_stars = []
        g_internal_distances = []

        nearby_star_indices = [i for i, x in enumerate(SNR_coord[g].separation(good_star_coord).deg < g_rad*2) if x == True]

        for i in range(len(nearby_star_indices)):
            star_list_index = good_indices[nearby_star_indices[i]]
            g_internal_stars.append([star_list['ra'][star_list_index], star_list['dec'][star_list_index]])
        ### Storing star values by appending an array for each variable
            GREEN_ID.append(g)
            REMNANT.append(g_SNR['SNR'])
            TELESCOPE.append(star_list['TELESCOPE'][star_list_index])
            FIELD.append(star_list['FIELD'][star_list_index])
            APOGEE_ID.append(star_list['APOGEE_ID'][star_list_index])
            RA.append(star_list['ra'][star_list_index])
            DEC.append(star_list['dec'][star_list_index])
            SNR.append(star_list['SNR'][star_list_index])
            TEFF.append(star_list['TEFF'][star_list_index])
            LOGG.append(star_list['LOGG'][star_list_index])
            M_H.append(star_list['M_H'][star_list_index])
            ALPHA_M.append(star_list['ALPHA_M'][star_list_index])
            RV.append(star_list['VHELIO_AVG'][star_list_index])
            ASPCAPFLAG.append(star_list['ASPCAPFLAG'][star_list_index])
            STARFLAG.append(star_list['STARFLAG'][star_list_index])
            EXTRATARG.append(star_list['EXTRATARG'][star_list_index])
        ### Storing the star's angular separation from the remnant center
            ANG_SEP.append(SNR_coord[g].separation(star_coord[star_list_index]).deg)
        ### Calculating & storing the star's physical separation on the plane of the sky
            physical_separation = SNR_dist[g]*np.tan(np.radians(SNR_coord[g].separation(star_coord[star_list_index]).deg))
            PHYS_SEP.append(physical_separation)
        ### Obtaining the current star's distance measurement/estimate
            star_dist = star_dis_list['r_est'][star_list_index]
        ### If there is an actual value, storing that value into the array for the .fits file
        ### and appending the count of stars with lengths
            if np.isnan(star_dist) == False:
                dist_count = dist_count + 1
        ### If no actual value, entering an arbitrary value of -9999
            else:
                star_dist = -9999
            g_internal_distances.append(star_dist)
            DIST.append(star_dist)
    print(g, len(nearby_star_indices))

(0, 0)
(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)
(11, 0)
(12, 78)
(13, 78)
(14, 78)
(15, 78)
(16, 78)
(17, 56)
(18, 56)
(19, 56)
(20, 56)
(21, 56)
(22, 56)
(23, 56)
(24, 56)
(25, 56)
(26, 56)
(27, 56)
(28, 56)
(29, 56)
(30, 56)
(31, 4)
(32, 4)
(33, 4)
(34, 4)
(35, 2)
(36, 2)
(37, 2)
(38, 2)
(39, 2)
(40, 2)
(41, 2)
(42, 2)
(43, 153)
(44, 153)
(45, 153)
(46, 153)
(47, 153)
(48, 5)
(49, 5)
(50, 5)
(51, 5)
(52, 5)
(53, 5)
(54, 5)
(55, 5)
(56, 5)
(57, 5)
(58, 2)
(59, 2)
(60, 6)
(61, 12)
(62, 12)
(63, 14)
(64, 14)
(65, 1)
(66, 1)
(67, 2)
(68, 3)
(69, 2)
(70, 17)
(71, 17)
(72, 17)
(73, 3)
(74, 3)
(75, 83)
(76, 83)
(77, 83)
(78, 4)
(79, 4)
(80, 4)
(81, 4)
(82, 0)
(83, 0)
(84, 0)
(85, 0)
(86, 7)
(87, 7)
(88, 0)
(89, 32)
(90, 39)
(91, 39)
(92, 39)
(93, 39)
(94, 14)
(95, 2)
(96, 2)
(97, 0)
(98, 0)
(99, 0)
(100, 0)
(101, 0)
(102, 0)
(103, 0)
(104, 0)
(105, 5)
(106, 5)
(107, 0)
(108, 18)
(109, 0)
(110, 1)
(111, 1)
(112, 0)
(113, 0)
(114, 0)
(115, 0)
(116, 17)
(117, 2)
(

### This section is for doing the calculation of the emission strength

In [9]:
### Importing the .fits file with binned median stellar residuals
from astropy.io import fits
import numpy as np

stellar_resid_dir = '/uufs/astro.utah.edu/common/uuastro/astrougrad/u1068803/SNR_Thesis/Stellar_Masks_V7.fits'
stellar_resid = fits.open(stellar_resid_dir)
### Obtaining info on bin range and spacing
bin_info = stellar_resid[0].header
delta_teff = bin_info['TEFF_spacing']
delta_logg = bin_info['LOGG_spacing']
delta_mh = bin_info['M_H_spacing']
teff_array = np.arange(bin_info['TEFF_MIN'], bin_info['TEFF_MAX'], delta_teff)
logg_array = np.arange(bin_info['LOGG_MIN'], bin_info['LOGG_MAX'], delta_logg)
mh_array = np.arange(bin_info['M_H_MIN'], bin_info['M_H_MAX'], delta_mh)
### Making an array with each imageHDU's TEFF/LOGG/M_H
bin_param_TEFF = []
bin_param_LOGG = []
bin_param_M_H = []
for temp_img in stellar_resid[1:]:
    temp_hdr = temp_img.header
    bin_param_TEFF.append(temp_hdr['bin_TEFF_min'])
    bin_param_LOGG.append(temp_hdr['bin_LOGG_min'])
    bin_param_M_H.append(temp_hdr['bin_M_H_min'])

In [10]:
### Function for shifting the spectrum back to the observed wavelength

def shift_obs_frame_v2(feature_range, wavelength, temp_spec, temp_err, temp_fit, radvelo):
    # Set range for interpolation
    range_for_interpolation = [feature_range[0]-10, feature_range[1]+10]
    feature_range_ind = (wavelength >= feature_range[0]) & (wavelength <= feature_range[1])
    wavelength_obs_frame = wavelength[feature_range_ind]
    n_pixels = np.sum(feature_range_ind)

    # Define array to hold observed-frame spectra, Nstars x Npixels
    residuals_obs_frame = np.zeros(n_pixels)
    #Normalized flux
    this_flux = temp_spec.copy()
    #Error
    this_ivar = 1./(temp_err**2)
    #ASPCAP fit
    this_fit = temp_fit.copy()
    #ASPCAP residual
    this_residual = np.divide(temp_spec, temp_fit)
    #Shift residual to observed frame
    this_observed_frame_wavelength = np.array([x + x*(radvelo/3e5)
                                               for x in wavelength])
    low_noise = (this_ivar > 1.0)
    range_for_interpolation_ind = (this_observed_frame_wavelength[low_noise] >= range_for_interpolation[0]) &\
                              (this_observed_frame_wavelength[low_noise] <= range_for_interpolation[1])
    f = interpolate.interp1d(this_observed_frame_wavelength[low_noise][range_for_interpolation_ind],
                             this_residual[low_noise][range_for_interpolation_ind])
    new_residual = f(wavelength_obs_frame)

    residuals_obs_frame = new_residual
    return wavelength_obs_frame, residuals_obs_frame

In [11]:
### Create master wavelength array
### First import the star's aspcap spectra
stellar_data_dir2 = '/uufs/chpc.utah.edu/common/home/sdss06/apogeework/'+\
                   'apogee/spectro/aspcap/t9/l31c/'
    
    
test_star = fits.open(stellar_data_dir2 + TELESCOPE[0].strip()  + '/' + \
                      FIELD[0].strip() + '/' + 'aspcapStar-t9-' + \
                      APOGEE_ID[0].strip() + '.fits')

wavelength_start = test_star[1].header['CRVAL1']
wavelength_logdisp = test_star[1].header['CDELT1']
num_wavelength = test_star[1].header['NAXIS1']
wavelength = 10**(wavelength_start + 
                  np.arange(0, wavelength_logdisp*num_wavelength, wavelength_logdisp))
test_star.close()

In [12]:
### Function for shifting the spectrum back to the observed wavelength
def shift_obs_frame_MedianResid(feature_range, wavelength, medresid_spec, medresid_err, radvelo):
    # Set range for interpolation
    range_for_interpolation = [feature_range[0]-10, feature_range[1]+10]
    feature_range_ind = (wavelength >= feature_range[0]) & (wavelength <= feature_range[1])
    wavelength_obs_frame = wavelength[feature_range_ind]
    n_pixels = np.sum(feature_range_ind)
    # Define array to hold observed-frame spectra, Nstars x Npixels
    residuals_obs_frame = np.zeros(n_pixels)
    #Median spectrum
    
    this_spec = medresid_spec.copy()
    #Error in median spectrum
    this_error = medresid_err.copy()
    #Shift both to observed frame
    this_observed_frame_wavelength = np.array([x + x*(radvelo/3e5)
                                               for x in wavelength])
    range_for_interpolation_ind = (this_observed_frame_wavelength >= range_for_interpolation[0]) &\
                              (this_observed_frame_wavelength <= range_for_interpolation[1])
    f = interpolate.interp1d(this_observed_frame_wavelength[range_for_interpolation_ind],
                             this_spec[range_for_interpolation_ind])
    g = interpolate.interp1d(this_observed_frame_wavelength[range_for_interpolation_ind],
                             this_error[range_for_interpolation_ind])
    new_medres_spec = f(wavelength_obs_frame)
    new_medres_error = g(wavelength_obs_frame)
    return new_medres_spec, new_medres_error

In [13]:
### A function for finding the index of an entry closest to a value
def find_closest_index(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx

In [14]:
### Treating each stellar spectrum with residual & median residual, and calculating emstr.
EMSTR = [] # Calculation of Emission Strength (W')
SPECWAVE = [] # Observed wavelength array
SPECRESID = [] # Reduced residual spectrum used to calculate EMSTR

import os
import math
from scipy import interpolate

count = 0
for i in range(len(APOGEE_ID)):
    temp_path = stellar_data_dir + TELESCOPE[i].strip() + '/' + \
                                          FIELD[i].strip() + '/' + 'aspcapStar-r12-' + \
                                          APOGEE_ID[i].strip() + '.fits'
    if os.path.isfile(temp_path):
        rv = RV[i]
        temp_star = fits.open(temp_path)
        temp_spec = temp_star[1].data
        temp_err = temp_star[2].data
        temp_fit = temp_star[3].data
        temp_star.close()
        temp_spec_mask = temp_spec.copy()
        temp_fit_mask = temp_fit.copy()
        ### Converting to observed wavelength given stellar RV
        obs_wave, obs_resid = shift_obs_frame_v2([16000, 16120], wavelength, temp_spec, temp_err, temp_fit, rv)
        ### Finding the appropriate stellar mask imageHDU
        temp_TEFF, temp_LOGG, temp_M_H = TEFF[i], LOGG[i], M_H[i]
        ind = (abs(bin_param_TEFF - temp_TEFF) <= delta_teff/2)\
        & (abs(bin_param_LOGG - temp_LOGG) <= delta_logg/2)\
        & (abs(bin_param_M_H - temp_M_H) <= delta_mh/2)
        true = [truth for truth,x in enumerate(ind) if x]
### This section is for the median residual spectra.
    ### They have to be rebinned to the observed wavelength before we can apply masks,
    ### because the interp function I use is not mask tolerant. 
    ### Checking to see if there actually is a median spectrum for this bin
        if len(true) != 0:
            if stellar_resid[true[0]+1].data[0].shape != ():
            ### The +1 accomodates for there being an empty header in the original .fits file
    ### Including a section for masking pixels based on the error in the median residual spectrum
                ### Convert temp_spec and the error in the median resid to an ma.array
                medianresid_spec = stellar_resid[true[0]+1].data[0]
                medianresid_error = stellar_resid[true[0]+1].data[1]
                ### Shift to observed wavelength
                obs_medres_spec, obs_medres_error = shift_obs_frame_MedianResid([16000, 16120], wavelength, medianresid_spec, medianresid_error, rv)
                ### Convert temp_spec and the error in the median resid to an ma.array
                medianresid_spec = np.ma.array(obs_medres_spec)
                medianresid_error = np.ma.array(obs_medres_error)
                ### Apply a mask to pixels where stdev > 0.1
                medianresid_spec_masked = np.ma.masked_where(medianresid_error >= 0.1, medianresid_spec)
                ### Convert observed residual array to ma.array
                obs_resid = np.ma.array(obs_resid)
                obs_resid = obs_resid / medianresid_spec#_masked
        ### Finding the standard deviation of a non-feature region nearby
        d_lambda = obs_wave[1] - obs_wave[0]
        nr_std = np.std(obs_resid[find_closest_index(obs_wave, 16089):find_closest_index(obs_wave, 16089)+100])
        nf_strength = 0
        nf_len = 0
        fr_strength = 0
        fr_len = 0
        ### Non-feature region
        for h in range(find_closest_index(obs_wave, 16089), find_closest_index(obs_wave, 16089)+100):
            nf_strength = nf_strength + obs_resid[h]*d_lambda
            nf_len = nf_len + 1
        ### Feature region
        for h in range(find_closest_index(obs_wave, 16060), find_closest_index(obs_wave, 16075)):
            fr_strength = fr_strength + obs_resid[h]*d_lambda
            fr_len = fr_len + 1
        ### Ratio of feature to non-feature regions
        nf_strength_norm = nf_strength / nf_len
        fr_strength_norm = fr_strength / fr_len
        strength_fnf_ratio = (fr_strength_norm/nf_strength_norm)
        EMSTR.append(strength_fnf_ratio)
        SPECWAVE.append(obs_wave)
        SPECRESID.append(obs_resid)
    count = count + 1
    if count == 500:
        print('Current Index: {} of {}'.format(i, len(APOGEE_ID)))
        count = 0
print('Done.')

/uufs/chpc.utah.edu/sys/installdir/python/2.7.11-c7/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: divide by zero encountered in divide
/uufs/chpc.utah.edu/sys/installdir/python/2.7.11-c7/lib/python2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in divide
/uufs/chpc.utah.edu/sys/installdir/python/2.7.11-c7/lib/python2.7/site-packages/ipykernel/__main__.py:45: RuntimeWarning: invalid value encountered in greater_equal
/uufs/chpc.utah.edu/sys/installdir/python/2.7.11-c7/lib/python2.7/site-packages/ipykernel/__main__.py:67: RuntimeWarning: divide by zero encountered in divide


Current Index: 499 of 3633
Current Index: 999 of 3633
Current Index: 1499 of 3633
Current Index: 1999 of 3633
Current Index: 2499 of 3633
Current Index: 2999 of 3633
Current Index: 3499 of 3633
Done.


### Saving everything into a .fits file

In [19]:
titles = ('GREEN_ID', 'REMNANT', 'TELESCOPE', 'FIELD', 'APOGEE_ID', 'RA', 'DEC', 'SNR',\
          'TEFF', 'LOGG', 'M_H', 'ALPHA_M', 'RV', 'ASPCAPFLAG', 'STARFLAG', 'EXTRATARG',\
          'ANG_SEP', 'PHYS_SEP', 'DIST', 'EMSTR', 'WAVE', 'RESID')
values = [GREEN_ID, REMNANT, TELESCOPE, FIELD, APOGEE_ID, RA, DEC, SNR,\
         TEFF, LOGG, M_H, ALPHA_M, RV, ASPCAPFLAG, STARFLAG, EXTRATARG,\
          ANG_SEP, PHYS_SEP, DIST, EMSTR, SPECWAVE, SPECRESID]

In [20]:
Stars_in_SNRs = QTable(values,names=titles)
print(len(Stars_in_SNRs))
Stars_in_SNRs.remove_rows(np.where(np.isnan(EMSTR)))
print(len(Stars_in_SNRs))
Stars_in_SNRs.write('Stars_in_SNRs_V7_SNRDistOnly.fits', format='fits', overwrite=True)

3633
3409


/uufs/chpc.utah.edu/sys/installdir/python/2.7.11-c7/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Warning: converting a masked element to nan.
  app.launch_new_instance()


In [18]:
Stars_in_SNRs

GREEN_ID,REMNANT,TELESCOPE,FIELD,APOGEE_ID,RA,DEC,SNR,TEFF,LOGG,M_H,ALPHA_M,RV,ASPCAPFLAG,STARFLAG,EXTRATARG,SEPARATION,DIST,EMSTR,WAVE [541],RESID [541]
int64,str11,str6,str8,str18,float64,float64,float32,float32,float32,float32,float32,float32,int32,int32,int16,float64,float64,float64,float64,float64
6,G003.7-00.2,apo25m,004+00,2M17550857-2558336,268.785724,-25.976004,171.012,3795.9172,0.18793404,-1.2292663,0.164595,-19.7641,0,2052,0,0.15691015090321533,-9999.0,0.9952135835698721,16000.074321874934 .. 16119.887256614487,0.9901914662616039 .. 0.9791739264983065
6,G003.7-00.2,apo25m,004+00,2M17551350-2553405,268.806263,-25.894608,805.419,3676.4866,1.0196086,0.24751368,-0.008956,40.0858,0,2048,0,0.07713582043117052,4336.8049622604,1.0089184143079375,16000.074321874934 .. 16119.887256614487,0.9818370274042441 .. 1.0136170043006656
6,G003.7-00.2,apo25m,004+00,2M17551732-2544346,268.822167,-25.74297,491.973,4134.839,1.5946302,0.17388368,0.007509999,-32.2803,0,1028,0,0.09605195049537255,3560.04407354525,1.0056138138144926,16000.074321874934 .. 16119.887256614487,1.0370531541086037 .. 0.9718443652244497
6,G003.7-00.2,apo25m,004+00,2M17554096-2548567,268.920682,-25.815777,375.054,4600.788,2.326705,0.14177369,-0.0280421,-19.3862,0,0,0,0.0588039805591465,1889.06057436862,1.0007005433011258,16000.074321874934 .. 16119.887256614487,1.0102410552482524 .. 0.9822750324998725
6,G003.7-00.2,apo25m,004+00,2M17554390-2551348,268.932953,-25.859673,163.322,3963.3823,1.3678782,0.21725368,0.0023679994,24.7352,0,0,0,0.07213588526490272,-9999.0,1.005121348762368,16000.074321874934 .. 16119.887256614487,1.0226094170149473 .. 0.9934937875076825
7,G003.8+00.3,apo25m,004+00,2M17524982-2519400,268.207584,-25.327805,166.056,4758.9727,2.5242836,0.22116369,-0.014909001,-14.1753,0,0,0,0.14022370905135398,3086.02912831832,1.0035486867057932,16000.074321874934 .. 16119.887256614487,1.0112785173174696 .. 1.0049867450405312
7,G003.8+00.3,apo25m,004+00,2M17532005-2544479,268.333552,-25.746647,321.967,4633.936,2.422195,0.21609369,-0.0035200007,-31.2194,0,0,0,0.2953810249136524,1412.32571035636,1.000229490308117,16000.074321874934 .. 16119.887256614487,1.023259156925947 .. 0.9795498435521748
7,G003.8+00.3,apo25m,004+00,2M17532090-2514343,268.337098,-25.242882,149.828,3754.101,0.5105824,-0.55391634,0.195545,9.73471,0,0,0,0.24411590076869477,-9999.0,1.0106533526275407,16000.074321874934 .. 16119.887256614487,0.9739010952373637 .. 1.017999990880088
7,G003.8+00.3,apo25m,004+00,2M17532805-2512049,268.366888,-25.201378,225.278,3794.3381,1.2613677,0.42381367,0.024829999,-171.435,0,2048,0,0.29303996977806546,4439.09335247462,0.9943118238343568,16000.074321874934 .. 16119.887256614487,0.9946496193929134 .. 1.026194638563746
